<a href="https://colab.research.google.com/github/kiyong21c/Must_Have_pytorch_DL/blob/main/20220926_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 데이터 살펴보기
 - 뉴욕 타임스 코멘트 데이터 사용

In [2]:
import pandas as pd
import os
import string

df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/CH10/ArticlesApril2017.csv")
print(df.columns)

Index(['abstract', 'articleID', 'articleWordCount', 'byline', 'documentType',
       'headline', 'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


 - headline : 사람이 직접 작성한 기사

In [3]:
df['headline'][0]

'Finding an Expansive View  of a Forgotten People in Niger'

In [8]:
df['headline'].shape # 886개의 헤드라인

(886,)

In [16]:
type(df.headline.values)

numpy.ndarray

## 학습용 데이터 만들기
 - 1. 구두점, 특수문자 제거
 - 2. 단어마다 고유번호 사전(BOW)
 - 3. 자연어 문장속 단어들을 고유번호로 바꿔 새로운 문장생성
 - 4. 두개의 단어와 다음에 올 단어를 반환하도록 데이터셋 객체 생성

In [ ]:
import numpy as np
import glob

from torch.utils.data.dataset import Dataset

# 학습에 사용할 데이터셋 객체 정의
class TextGeneration(Dataset):
    # 텍스트 → 소문자, 특수문자 제거하는 함수
    def clean_text(self, txt): # txt로 받을것 : 기사문자열
        txt = "".join(v for v in txt if v not in string.punctuation) # 리스트에 들어있는 각 문자열을 합침(빈공백으로), 특수문자 제외
        # string.punctuation : 특수문자모아놓은 리스트
        txt = txt.lower()
        return txt
    
    def  __init__(self):
        all_headlines = []

        # 모든 헤드라인의 텍스트를 불러옴
        for filename in glob.glob("/content/drive/MyDrive/Colab Notebooks/data/CH10/*.csv"):
            if 'Articles' in filename: # 기사에 해당되는 파일만 선별해서 df으로
                article_df = pd.read_csv(filename)
                
                # 데이터셋의 headline값을 all_headlines에 추가
                all_headlines.extend(list(article_df['headline'].values)) # .values : array
                # list.extend(iterable) : iterable객체(그대로가 아닌) 요소들을 list에 넣는다
                break

        # headline중 Unknown값 제거
        all_headlines = [h for h in all_headlines if h!= "Unknown"]

        # 구두점 제거 및 전처리가 된 문장들을 리스트로 반환
        self.corpus = [self.clean_text(x) for x in all_headlines]
        self.BOW = {} # {"단어" : 고유숫자}

        # 모든 문장의 단어를 추출해 고유 번호 지정
        for line in self.corpus: # line : 전처리된 각 헤드라인 문장
            for word in line.split(): # word : 단어 단위
                if word not in self.BOW.keys():
                    self.BOW[word] = len(self.BOW.keys())






In [36]:
a = ['a', 'b', 'c', '!', 'd']
"".join(a)

'abc!d'

In [37]:
b = "".join(v for v in a if v not in string.punctuation) # string.punctuation : 특수문자모아놓은 리스트
b

'abcd'

In [39]:
len(b.split())

1

In [29]:
b= ['a','b','c']
c = [['ab','cd']]

In [30]:
b.extend(c)

In [40]:
df['headline'][0]

'Finding an Expansive View  of a Forgotten People in Niger'

In [43]:
"".join(v for v in list(df['headline'][0]) if v not in string.punctuation)

'Finding an Expansive View  of a Forgotten People in Niger'

In [1]:
e = {'e':[1,2,3,4]}


In [4]:
e['e']

[1, 2, 3, 4]